## Direct location with Sentinel-3 OLCI sensor 

This notebook shows examples on direct location performed using OLCI sensor and using DEM GETAS zarr format.

In [ ]:
import os
import json

import numpy as np

from asgard.core.time import TimeRef
from asgard.models.time import TimeReference
from asgard.sensors.sentinel3 import S3OLCIGeometry

from asgard_legacy_drivers.drivers.geojson_util import to_points
from asgard_legacy_drivers.drivers.sentinel_3_legacy import S3LegacyDriver

### Setup variables and configure S3OLCIGeometry

In [ ]:
# these directories are required 
TEST_DIR = os.environ.get("TEST_DIR", "../tests")
ASGARD_DATA = os.environ.get("ASGARD_DATA", "/data/asgard")
GETAS_PATH = os.path.join(ASGARD_DATA, "ADFstatic", "S0__ADF_GETAS_20000101T000000_21000101T000000_20240529T142617.zarr")

In [ ]:
# setting paths to the orbit and calibration files 
orbit_file = os.path.join(
        TEST_DIR,
        "resources/S3/FRO",
        "S3A_OPER_MPL_ORBRES_20220510T000000_20220520T000000_0001.EOF",
    )
calibration_file = os.path.join(TEST_DIR, "resources/S3/OLCI/CAL", "OL_1_CAL_AX.nc")

In [ ]:
# generating image coordinates
img_coords = np.zeros((10, 74, 2), dtype="int32")
for row in range(10):
    for col in range(74):
        img_coords[row, col, 0] = col
        img_coords[row, col, 1] = row

In [ ]:
# extracting pointing vectors 
pointing_vectors = S3LegacyDriver.olci_pointing_angles(calibration_file)

In [ ]:
# extracting thermoelastic grids
thermoelastic = S3LegacyDriver.s3_thermoelastic_tables(calibration_file, group="thermoelastic_model_EO")

In [ ]:
# extarcting FRO orbit from 2022-05-10
fro_20220510 =  S3LegacyDriver.read_orbit_file(orbit_file)

In [ ]:

# setting frames
frames = {
    "offsets": np.array(
        [8168.024560769051 + 0.000001 * k for k in range(100)], dtype="float64"
    ),
}

# instantiating a navatt 
navatt_gps = np.load(os.path.join(TEST_DIR, "resources/sample_timestamps_gps.npy"))
navatt_oop = np.load(os.path.join(TEST_DIR, "resources/sample_oop.npy"))
navatt_orb = S3LegacyDriver.read_orbit_file(
    os.path.join(TEST_DIR, "resources/sample_orbit.xml")
)
navatt_att = S3LegacyDriver.read_attitude_file(
    os.path.join(TEST_DIR, "resources/sample_attitude.xml")
)
time_model = TimeReference()
navatt_att["times"]["GPS"] = {}
navatt_att["times"]["GPS"]["offsets"] = np.array([time_model.convert(time, ref_in=TimeRef.UTC, ref_out=TimeRef.GPS) for time in navatt_att["times"]["UTC"]["offsets"]])
navatt_att["time_ref"] = "GPS"

# setting config for olci product
config = {
    "sat": "SENTINEL_3",
    "orbit_aux_info": {
      "orbit_state_vectors": [fro_20220510],
    },
    "resources" : 
    {
        "dem_path": GETAS_PATH,
        "dem_type": "ZARR_GETAS",
    },
    "pointing_vectors": pointing_vectors,
    "thermoelastic": thermoelastic,
    "frame": {"times": frames},
    "navatt": {
        "orbit": navatt_orb,
        "attitude": navatt_att,
        "times": {
            "offsets": navatt_gps,
        },
        "oop": navatt_oop,
    },
}


In [ ]:
# instantiation of the geometry
olci =  S3OLCIGeometry(**config)

### Direct location with GETAS zarr

In [ ]:
# running direct locations using a pre-generated coordinates grid img_coords
gnd_points, _ = olci.direct_loc(img_coords, geometric_unit="C2")

### Visualize direct location on map 

In [ ]:
# converting ground positions gnd_point to GEOJson MultiPoint at indicated path
# create live directory if it doesn't exist
live_dir = os.path.join(TEST_DIR, "outputs/live")
if not os.path.exists(live_dir):
    os.makedirs(live_dir)

path_points = os.path.join(TEST_DIR, "outputs/live/olci_points.geojson")
# conversion to GeoJson
to_points(gnd_points, path_points)

In [ ]:
# opening file and extracting coordinates
with open(path_points, 'r') as f:
    json_data_point = json.load(f)
coordinates_points = json_data_point['features'][0]['geometry']['coordinates']

In [ ]:
import folium

# initializing the locations map
m = folium.Map(location=coordinates_points[0], zoom_start=3)
# iterate through the coordinates and add them as circle-shaped markers to the map
for feature in coordinates_points:
    coordinates = feature[::-1]
    marker = folium.CircleMarker(location=coordinates, radius=5, color='blue', fill=True, fill_color='blue')
    marker.add_to(m)

In [ ]:
# saving the resulting map
m.save('map_olci_points.html')

In [ ]:
# plotting our GeoJSON locations 
m